# Required Packages

In [4]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install openai
%pip install tiktoken
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Imports

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Load and split into chunks the pdf data

In [6]:
loader=PyPDFDirectoryLoader("pdfs")

In [7]:
data=loader.load()

In [8]:
data[0]

Document(page_content='Orca-Math: Unlocking the potential of\nSLMs in Grade School Math\nArindam Mitra∗, Hamed Khanpour, Corby Rosset, Ahmed Awadallah\nMicrosoft Research\nAbstract\nWe show that an SLM can reach ∼87%pass@1 on GSM8K while trained on only 200K\nsynthetic math problems. Mathematical word problem-solving has long been recognized as\na complex task for small language models (SLMs). A recent study hypothesized that the\nsmallest model size, needed to achieve over 80% accuracy on the GSM8K benchmark, is\n34 billion parameters. To reach this level of performance with smaller models, researcher\noften train SLMs to generate Python code or use tools to help avoid calculation errors.\nAdditionally, they employ ensembling, where outputs of up to 100 model runs are combined\nto arrive at a more accurate result. Result selection is done using consensus, majority vote\nor a separate a verifier model used in conjunction with the SLM. Ensembling provides a\nsubstantial boost in accurac

In [9]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [10]:
text_chunks=text_splitter.split_documents(data)

In [11]:
print(text_chunks[0].page_content)

Orca-Math: Unlocking the potential of
SLMs in Grade School Math
Arindam Mitra∗, Hamed Khanpour, Corby Rosset, Ahmed Awadallah
Microsoft Research
Abstract
We show that an SLM can reach ∼87%pass@1 on GSM8K while trained on only 200K
synthetic math problems. Mathematical word problem-solving has long been recognized as
a complex task for small language models (SLMs). A recent study hypothesized that the
smallest model size, needed to achieve over 80% accuracy on the GSM8K benchmark, is


In [12]:
len(text_chunks)

93

In [13]:
# last chunk
print(text_chunks[92].page_content)

Chen. Mammoth: Building math generalist models through hybrid instruction tuning. arXiv
preprint arXiv:2309.05653 , 2023.
[44]Yifan Zhang, Yifan Luo, Yang Yuan, and Andrew Chi-Chih Yao. Templatemath: Syntactic
data generation for mathematical problems, 2024.
[45]Chunting Zhou, Pengfei Liu, Puxin Xu, Srini Iyer, Jiao Sun, Yuning Mao, Xuezhe Ma, Avia
Efrat, PingYu, LiliYu, etal. Lima: Lessismoreforalignment. arXiv preprint arXiv:2305.11206 ,
2023.
14


In [14]:
key=os.getenv("OPENAI_API_KEY")

In [15]:
embedding=OpenAIEmbeddings()

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Embedding example

In [16]:
# number of features
len(embedding.embed_query("Hi there what is up?"))

1536

In [17]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV=os.getenv("PINECONE_API_ENV")

In [18]:
from pinecone import Pinecone

In [19]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("test")

In [20]:
index_name="test"

In [21]:
docsearch = PineconeVectorStore.from_documents(text_chunks, embedding, index_name=index_name)

In [22]:
query = "Who are the Autors"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Blecher, Cristian Canton Ferrer, Moya Chen, Guillem Cucurull, David Esiobu, Jude Fernandes,
Jeremy Fu, Wenyin Fu, Brian Fuller, Cynthia Gao, Vedanuj Goswami, Naman Goyal, Anthony
Hartshorn, Saghar Hosseini, Rui Hou, Hakan Inan, Marcin Kardas, Viktor Kerkez, Madian
Khabsa, Isabel Kloumann, Artem Korenev, Punit Singh Koura, Marie-Anne Lachaux, Thibaut
Lavril, Jenya Lee, Diana Liskovich, Yinghai Lu, Yuning Mao, Xavier Martinet, Todor Mihaylov,


In [23]:
llm=OpenAI()

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [24]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [25]:
query = "Who are the Autors"

In [26]:
qa.run(query)

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThe authors listed in the given context are Blecher, Cristian Canton Ferrer, Moya Chen, Guillem Cucurull, David Esiobu, Jude Fernandes, Jeremy Fu, Wenyin Fu, Brian Fuller, Cynthia Gao, Vedanuj Goswami, Naman Goyal, Anthony Hartshorn, Saghar Hosseini, Rui Hou, Hakan Inan, Marcin Kardas, Viktor Kerkez, Madian Khabsa, Isabel Kloumann, Artem Korenev, Punit Singh Koura, Marie-Anne Lachaux, Thibaut Lavril, Jenya Lee, Diana Liskovich, Yinghai Lu, Yuning Mao, Xavier Martinet, Todor Mihaylov, Saulnier, Lélio Renard Lavaud, Pierre Stock, Teven Le Scao, Thomas Wang, Timothée Lacroix, and William El Sayed.'

In [28]:
import sys
while True:
    user_input=input(f"Input Prompt: ")
    if user_input == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        continue
    result=qa({'query': user_input})
    print(f"Answer: {result['result']}")

Answer: 
The main topic is evaluating a student's answer to a word problem in math.
Exiting


SystemExit: 

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
